In [11]:
from skimage.draw import random_shapes
from scipy.spatial import distance
from PIL import Image
import numpy as np

In [12]:
class Artist:
    def __init__(self, img_path):
        """
        :param img_path: path to input image
        """
        img = Image.open(img_path)
        img = np.array(img)
        # algorithm works correctly only for input images 512 * 512 pixels
        assert img.shape == (512, 512, 3)
        print('Use {} as input picture'.format(img_path))
        self.fitness_vector = img.reshape((img.shape[0] * img.shape[1] * img.shape[2]))

In [13]:
    def reset(self, img_path):
        """
        Reset input image
        :param img_path: path to input image
        """
        img = Image.open(img_path)
        img = np.array(img)
        self.fitness_vector = img.reshape((img.shape[0] * img.shape[1] * img.shape[2]))

In [14]:
    def _fitness(self, x) -> int:
        """
        Calculate fitness as pixel-wise euclidean distance
        :param x: current state of the new image
        :return: euclidean distance between two images
        """
        return distance.euclidean(x, self.fitness_vector)

In [15]:
    @staticmethod
    def _initial_population(population_size) -> np.array:
        """
        :param population_size: init population size
        :return: np.array with shape (population_size, 512*512*3)
        """
        population = random_shapes((512, 512), max_shapes=2, allow_overlap=True, max_size=30,
                                   intensity_range=((0, 255), (0, 255), (0, 255)))[0].reshape(512 * 512 * 3)
        for i in range(population_size - 1):
            population = np.vstack((population, random_shapes((512, 512), max_shapes=2, max_size=30,
                                                              allow_overlap=True,
                                                              intensity_range=((0, 255), (0, 255), (0, 255)))[
                0].reshape(
                512 * 512 * 3).T))

        return population

In [16]:
    def _selection(self, population) -> np.array:
        """
        Select top 2 images from population
        :return: top 2 images from population
        """
        costs = np.apply_along_axis(self._fitness, 1, population)
        return np.array([population[np.argsort(costs)[:2][0]], population[np.argsort(costs)[:2][1]]])

In [17]:
    def _crossover(self, best_examples):
        """
        Copy best images
        :param best_examples:
        :return: x4 top images
        """
        return np.vstack(([best_examples] * 4))

In [18]:
    def _mutation(self, population, learning_rate):
        """
        Throw random shape on the pictures
        :param population:
        :param learning_rate: regularizes shape sice
        :return:
        """
        mutations = random_shapes((512, 512), max_shapes=1, max_size=learning_rate, allow_overlap=True,
                                  intensity_range=((0, 255),))[0].reshape(512 * 512 * 3)
        for i in range(population.shape[0] - 1):
            mutations = np.vstack(
                (mutations, random_shapes((512, 512), max_shapes=1, max_size=learning_rate,
                                          allow_overlap=True,
                                          intensity_range=((0, 255), (0, 255), (0, 255)))[0].reshape(
                    512 * 512 * 3).T))

        return population + mutations + 1

In [19]:
    def draw(self, number_of_epochs=1000, init_population_size=10, drafts: int = False, learning_rate=100, logs=False):
        """
        :param number_of_epochs
        :param init_population_size: amount of rows in initial population
        :param drafts: print drafts or not
        :param learning_rate: controls shape sizes
        :return: final image as array ((512, 512, 3))
        """
        population = self._initial_population(init_population_size)
        top = self._selection(population)
        for i in range(1, number_of_epochs):
            top = self._selection(population)
            children = self._crossover(top)
            noise_children = self._mutation(children, learning_rate)
            population = np.vstack((top, noise_children))

            if drafts:
                if i % drafts == 0:
                    my_matrix = top[0].reshape((512, 512, 3))
                    img = Image.fromarray(np.uint8(my_matrix))
                    img.show()

            if learning_rate >= 20:
                learning_rate -= 1

            if logs:
                print('Iteration : {}'.format(i))

        return top[0].reshape((512, 512, 3))

In [10]:
artist = Artist('put_in.jpg')
putin = artist.draw(number_of_epochs=1000, drafts=500, logs=True)
img = Image.fromarray(np.uint8(putin))
img.save('my_putin')

FileNotFoundError: [Errno 2] No such file or directory: 'put_in.jpg'